# Referencias
Me baso en dos fuentes distintas, una para el algoritmo completo y otra para el tratamiento de la cola:
- Marsaglia, G., & Tsang, W. W. (2000). The ziggurat method for generating random variables. Journal of Statistical Software, 5(8), 1–7.
- Marsaglia, G. (1963). Generating a variable from the tail of the normal distribution (Mathematical Note No. 322). Boeing Scientific Research Laboratories.

# Procedimiento
## Intención
Se implementará un algoritmo Ziggurat y una versión modificada para tratar mejor la cola de la función de densidad que estemos generando, ambas versiones se compararán con otras dos funciones en la generación de una exponencial y luego usaremos estas versiones de Ziggurat para hacer la tabla de densidad de una distribución normal estándar.

## Descripción de los algoritmos

### Algoritmo de Ziggurat Estándar
#### Precálculo
Partimos de nuestro conjunto $C$ que es el área bajo la curva $y=f(x)=\lambda e^{- \lambda x}$. La idea es generar un área $Z$$ tal que $Z \supset C$, este área la conformamos por un conjunto $N$ de cajas con mismo área $v$.
Para maximizar la eficiencia se busca que $N$ sea un múltiplo de 2, de tal manera que se pueda aprovechar la estructura binaria de los ordenadores, y poder asi tomar valores truncando ciertos valores, pero nos estamos adelantando. La idea es que las cajas sean 32, 64, 128, 256, etc. En este caso usaremos 255 para la exponencial concretamente, para la normal se usa 256.

Para generar las cajas en base a nuestro N debemos seguir una relación concreta: TODAS las cajas deben tener área $v$. Definimos las cajas tal que:
$$x_0 = 0 < x_1 < x_2 < ... < x_N = r$$
$$v = x_i [f(x_{i-1}) - f(x_i)] \qquad i= 1, 2, ..., N$$
Esto se define así para todas las cajas, pero para el caso $x_N = r$ no solo contiene la caja, contiene además el área de la cola de la función decreciente, tal que:
$$v = r f(r) +  \int_{r}^{\infty} f(r), dx $$

##### Cálculo de r
Como se puede ver, aún teniendo v dado, este depende de r, por lo que los investigadores nos han proporcionado la función $z(r)$ que nos permite ver si el valor de $r$ que estamos utilizando cumple con las definicinoes y todas las cajas que genera tiene área v. Esta función se define como:

*Algorithm: $z(r)$ para el cálculo del valor óptimo de $r$*  

*Input:*  
$f(x)$ (densidad decreciente),  
$f^{-1}(y)$ (inversa de $f$),  
$N$ (número de rectángulos).  

*Output:*  
$z(r)$ (valor que debe anularse para obtener el $r$ correcto).  

---

1. $( x_N \leftarrow r )$

2. $( v \leftarrow r,\quad f(r) + \displaystyle\int_{r}^{\infty} f(x),dx )$

3. *for* $i = N-1, N-2, \dots, 1$ 
   
   *do:*  
   $\quad x_i \leftarrow f^{-1}\!\left(f(x_{i+1}) + \dfrac{v}{x_{i+1}}\right ) $

4. *return* $z(r) = v - \left(x_1 - x_1 f(x_1)\right) $


---

##### Valores por defecto y versiones empleadas

En este ejercicio, como vamos a comparar con la **distribución exponencial** y más adelante generaremos la **distribución normal estándar**, **no calcularemos el valor de \( r \)** ni el área $v$** de forma numérica**.  
Para simplificar el trabajo, utilizaremos directamente los valores de referencia proporcionados por Marsaglia y Tsang (2000), que son los que usan la mayoría de implementaciones modernas del método de Ziggurat.

- **Exponencial estándar (λ = 1)**  
  $f(x) = e^{-x}, \quad x \ge 0 $  
  Se trata de la versión clásica de la exponencial decreciente.  
  En este caso se utilizan **255 rectángulos (N = 255)**, con los valores:
  $$
  r = 7.69711747013104972, \qquad
  v = 0.003949659822581557
  $$
  La eficiencia del método con estos parámetros es del **98.9 %**.


- **Normal estándar (mitad positiva)**  
  $ f(x) = e^{-x^2/2}, \quad x \ge 0 $  
  Se usa únicamente la mitad positiva de la normal estándar (sin constante de normalización).  
  Para obtener la distribución completa $ N(0,1) $, basta con asignar un signo aleatorio $ \pm1 $ con probabilidad $ 1/2 $.  
  En este caso se utilizan **256 rectángulos (N = 256)**, con los valores:
  $$
  r = 3.6541528853610088, \qquad
  v = 0.00492867323399
  $$
  La eficiencia del método con estos parámetros es del **99.33 %**.


Estos valores provienen directamente del trabajo original de *Marsaglia y Tsang (2000), “The Ziggurat Method for Generating Random Variables”*, y nos permiten concentrarnos en la comprensión y comparación del método sin añadir complejidad innecesaria en el cálculo de $r $ y $ v $.

##### Cálculo de las cajas
En este punto ya tenemos r, ya conocemos la relación entre una caja y la anterior y conocemos las restricciones (v) de estas cajas, solo tenemos que invertir la funcion para conseguir calcular todas las cajas:
$$f(x_{i-1}) = f(x_i) + \frac{v}{x_i}$$
$$x_{i-1} = f^{-1}(f(x_i) + \frac{v}{x_i})$$

#### Algoritmo Generador

Ahora que ya tenemos las cajas (y suponemos que tienes un generador de la distribuión uniforme funcional) podemos mostrar como es el algoritmo de Ziggurat a la hora de generar distribuciones.

---

*Algorithm: Generación de una variable aleatoria mediante el método de Ziggurat (con tratamiento de colas de Marsaglia, 1963)*  

*Input:*  
$f(x)$ (función de densidad decreciente),  
$\{x_i\}, \{f_i\}$ (tablas precalculadas),  
$r, v, N$ (parámetros del ziggurat).  

*Output:*  
$x$ (valor aleatorio con la distribución objetivo).  

---

1. Generar un entero aleatorio de 32 bits $j$.

2. Obtener el índice de caja:
   $$
   i \leftarrow j \; \&\; (N-1)
   $$
   (usa los bits bajos de $j$).

3. Calcular el candidato:
   $$
   x \leftarrow |j| \cdot w[i]
   $$
   donde $w[i] = \dfrac{x_i}{2^{32}}$.

4. **Comprobación rápida:**  
   Si $|j| < k[i]$ entonces **aceptar directamente** y devolver $x$  
   (se encuentra en la parte segura del rectángulo, bajo la curva).

5. **Si no se cumple:**
   - **Caso cola (i = 0):**
     1. Generar $u_1, u_2 \sim U(0,1)$.
     2. Calcular
        $$
        x \leftarrow r + \frac{-\ln(u_1)}{r}
        $$
     3. Aceptar si
        $$
        u_2 < e^{-\frac{1}{2}(x^2 - r^2)}.
        $$
        Si no, repetir desde el paso 5.

   - **Caso caja interna (i > 0):**
     1. Calcular $f(x) = e^{-x^2/2}$ (o la correspondiente para la distribución).
     2. Aceptar si
        $$
        f(x) > f_i + \dfrac{f_{i-1} - f_i}{x_i - x_{i-1}}(x - x_i)
        $$
        (el punto está bajo la curva).  
        Si no, repetir desde el paso 1.

6. Para la **normal**, asignar signo aleatorio:
   $$
   x \leftarrow \pm x \quad \text{con prob. } 0.5
   $$

7. Devolver $x$.



# Implementación Problema 1
 Hay que tener instalado numpy y scipy para poder ejecutar este codigo
 Tenemos que generar una aproximación de la tabla de distribución de la normal estándar. Para ello haremos la generacion de una gran numero de valores para la normal y calcularemos las probabilidades empiricamente. Es decir, una vez con los numeros generados aleatoriamente vamos a calcular su probabilidad para los valores de una tabla ya calculada con pasos de 0.1. Finalmente tomaremos esta tabla ya calculada y calcularemos cuanto nos hemos alejado de la tabla teorica

In [ ]:
#pip install numpy

  Using cached numpy-2.3.4-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.4-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install scipy

  Using cached scipy-1.16.3-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
Using cached scipy-1.16.3-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Note: you may need to restart the kernel to use updated packages.


Empecemos escogiendo nuestro generador uniforme en este caso tomaremos Mersenne Twister por su fiabilidad y por su periodo tan grande, que está implementado en la libreria numpy

In [7]:
import numpy as np

# Crear generador basado en Mersenne Twister
mt = np.random.MT19937(seed=43)
rng = np.random.Generator(mt)

print([rng.random() for _ in range(10)])


[0.6057220977812905, 0.9245321580950842, 0.6383927532488509, 0.11361476091492895, 0.7447303871101164, 0.42490588783923344, 0.46765813308716786, 0.018731721960731584, 0.0075794920916104624, 0.7952568777367439]


Ahora calcularemos las cajas, acorde al paper no nos esforzaremos en encontrar un v y un r nuevo, así que simplemente haremos el cálculo con las variables que se nos ofrecen:
$$r = 128, \qquad r = 3.6541528853610088, \qquad v = 0.00492867323399$$

Se utilizan 128 cajas porque aqí podemos utilizar los primeros 7 bits del numero aleatorio de 32 bits para elegir la caja, luego con el bit más significativo lo utilizaremos para saber el signo de este.

Para generar las cajas utilizamos la fórmula


In [ ]:
import numpy as np

def ziggurat_norm_boxes(n_layers=128, r=3.442619855899, v=9.91256303526217e-3):
    """
    Calcula los bordes x_i de las cajas del ziggurat para la normal estándar (lado positivo).

    Parámetros
    ----------
    n_layers : int
        Número de capas (por convención 128 para la normal simétrica).
    r : float
        Límite superior (x_N) según el paper de Marsaglia & Tsang (2000).
    v : float
        Área común de cada rectángulo según el mismo paper.

    Devuelve
    --------
    np.ndarray
        Array con los bordes x_i (de longitud n_layers), desde x_0=0 hasta x_N=r.
    """
    # Función densidad sin normalizar (lado positivo)
    f = lambda x: np.exp(-0.5 * x * x)

    # Array de bordes (x_0, ..., x_N)
    x = np.zeros(n_layers, dtype=np.float64)
    x[-1] = r  # último borde

    # Recorremos hacia abajo desde la última capa hasta la primera
    for i in range(n_layers - 2, -1, -1):
        # f_inv(y) = sqrt(-2 ln y)
        y = f(x[i + 1]) + v / x[i + 1]
        x[i] = np.sqrt(-2.0 * np.log(y))

    return x


# Ejemplo de uso
if __name__ == "__main__":
    x_vals = ziggurat_norm_boxes()
    print("Primeros bordes:", x_vals[:5])
    print("Últimos bordes:", x_vals[-5:])
    print("Área de la última caja (comprobación):", 
          x_vals[-1] * (np.exp(-0.5*x_vals[-2]**2) - np.exp(-0.5*x_vals[-1]**2)))


Para comprobar que mi distribución sigue la normal aplicando el KS con la N(0,1), y haremos media = 0 con chi cuadrado y varianza = 1 con t student

Perfecto 😄 — no pasa nada, te explico **cómo se hacen los tres contrastes** (t, χ² y KS) desde cero, paso a paso y con las fórmulas que normalmente se piden en un TFG o informe técnico, para que puedas escribirlos sin depender de bibliografía externa.

---

## 🧠 1. Test t para la media

Sirve para comprobar si la media de tu muestra ( \bar{x} ) es **igual a un valor teórico** (en tu caso, 0).

### 🔹 Hipótesis

$
H_0: \mu = 0 \quad\quad H_1: \mu \ne 0
$

### 🔹 Estadístico

$
t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}
$
donde:

* ( $\bar{x}$ ): media muestral
* ( $s$ ): desviación estándar muestral
* ( $n$ ): tamaño de la muestra
* ( $\mu_0 = 0$ ): valor teórico bajo ( $H_0$ )

Bajo ( $H_0$ ), ( $t \sim t_{n-1}$ ).

### 🔹 Regla de decisión

* Calculas el **p-valor** a partir de la distribución t con ( n-1 ) grados de libertad.
* Si ( p > 0.05 ), **no rechazas ( H_0 )** → la media es compatible con 0.

### 🔹 Ejemplo en Python

```python
from scipy.stats import ttest_1samp
t, p = ttest_1samp(samples, 0)
print(f"t = {t:.4f}, p = {p:.4f}")
```

---

## ⚙️ 2. Test χ² para la varianza

Permite comprobar si la **varianza** muestral ( s^2 ) es igual a un valor teórico (1, para la normal estándar).

### 🔹 Hipótesis

[
H_0: \sigma^2 = 1 \quad\quad H_1: \sigma^2 \ne 1
]

### 🔹 Estadístico

[
\chi^2 = \frac{(n - 1) s^2}{\sigma_0^2}
]

donde:

* ( \sigma_0^2 = 1 ): varianza teórica
* Bajo ( H_0 ), ( \chi^2 \sim \chi^2_{n-1} ).

### 🔹 Regla de decisión

* Si ( \chi^2 ) está **dentro del intervalo**:
  [
  \chi^2_{\alpha/2, n-1} < \chi^2 < \chi^2_{1-\alpha/2, n-1}
  ]
  no se rechaza ( H_0 ).
* Equivalente: ( p > 0.05 ) → varianza compatible con 1.

### 🔹 Ejemplo en Python

```python
from scipy.stats import chi2
n = len(samples)
s2 = np.var(samples, ddof=1)
chi2_stat = (n-1)*s2/1
p = 2 * min(chi2.cdf(chi2_stat, n-1), 1 - chi2.cdf(chi2_stat, n-1))
print(f"chi² = {chi2_stat:.4f}, p = {p:.4f}")
```

---

## 📈 3. Test de Kolmogórov–Smirnov (bondad de ajuste)

Sirve para comprobar si los datos siguen una distribución teórica completa (en tu caso, ( N(0,1) )).

### 🔹 Hipótesis

[
H_0: F(x) = \Phi(x) \quad\quad H_1: F(x) \ne \Phi(x)
]

donde:

* ( F(x) ): CDF empírica de tu muestra
* ( \Phi(x) ): CDF teórica de la normal estándar

### 🔹 Estadístico

[
D = \max_x |F_n(x) - \Phi(x)|
]
Bajo ( H_0 ), el valor de ( D ) sigue una distribución conocida (dependiente de ( n )).
En la práctica, se compara el **p-valor** con 0.05.

### 🔹 Regla de decisión

* Si ( p > 0.05 ): la muestra **no difiere significativamente** de una normal estándar.
* Si ( p ≤ 0.05 ): se **rechaza ( H_0 )** → la forma no coincide con la teórica.

### 🔹 Ejemplo en Python

```python
from scipy.stats import kstest
D, p = kstest(samples, 'norm')
print(f"D = {D:.4e}, p = {p:.4f}")
```

⚠️ Solo aplica directamente si tus datos son ( N(0,1) ); si no, normalízalos antes:

```python
normalized = (samples - np.mean(samples)) / np.std(samples, ddof=1)
D, p = kstest(normalized, 'norm')
```

---

## ✅ 4. Interpretación conjunta

| Test         | Propósito    | No rechazo H₀ significa…  |
| ------------ | ------------ | ------------------------- |
| t de Student | Media = 0    | No hay sesgo en el centro |
| χ²           | Varianza = 1 | Escalado correcto         |
| KS           | Forma normal | La distribución es normal |

Si los tres tests no rechazan ( H_0 ), tu generador **cumple con ( N(0,1) )**.

---

¿Quieres que te monte este mismo contenido en bloque Markdown bonito (como para incluirlo en tu memoria, con fórmulas LaTeX y texto explicativo breve debajo de cada contraste)?


Perfecto 👌 Aquí tienes un **resumen completo en formato Markdown**, listo para copiar a tu memoria o README, con foco en el **experimento empírico**, su justificación teórica y las métricas clave para validar tu implementación del Ziggurat.

---

## Validación empírica del algoritmo Ziggurat

### 🧩 Objetivo

El propósito del experimento es **evaluar la calidad estadística y computacional** del algoritmo de generación de números normales estándar implementado mediante el método de **Ziggurat**, verificando que las muestras generadas:

* Reproduzcan correctamente la distribución normal teórica ( \mathcal{N}(0,1) ).
* Presenten errores decrecientes al aumentar el tamaño de muestra.
* Mantengan eficiencia y estabilidad al depender de un generador uniforme de alta calidad.

---

### 🧠 Fundamento teórico

La **función de distribución empírica (EDF)** o **empirical distribution function** permite estimar la función de distribución teórica ( F(x) ) a partir de una muestra ( x_1, \dots, x_n ):

[
\hat{F}*n(x) = \frac{1}{n} \sum*{i=1}^{n} \mathbf{1}{x_i \le x}
]

donde ( \mathbf{1}{\cdot} ) es la función indicadora.
Según Taboga (2021), ( \hat{F}_n(x) ) es un **estimador insesgado y consistente** de ( F(x) ), y su varianza disminuye con el tamaño muestral ( n ):

[
\text{Var}[\hat{F}_n(x)] = \frac{F(x),[1 - F(x)]}{n}
]

De acuerdo con el **teorema de Glivenko–Cantelli**, la EDF converge **uniformemente** a la CDF teórica cuando ( n \to \infty ).

> **Referencia:**
> Taboga, M. (2021). *Empirical distribution*. StatLect.
> Disponible en: [https://www.statlect.com/asymptotic-theory/empirical-distribution](https://www.statlect.com/asymptotic-theory/empirical-distribution)

---

### ⚙️ Diseño experimental

El experimento se divide en **dos fases**:

#### 1. Validación estadística

Se generan muestras ( z_1, \dots, z_n \sim \mathcal{N}(0,1) ) usando el algoritmo Ziggurat con distintos tamaños de muestra:

* **Muestra pequeña:** ( n = 10^4 )
* **Muestra grande:** ( n = 10^6 ) o superior

Para cada conjunto:

1. Calcular la **media** y **varianza empíricas**:
   [
   \bar{z} = \frac{1}{n}\sum z_i, \quad s^2 = \frac{1}{n-1}\sum (z_i - \bar{z})^2
   ]

2. Estimar la **CDF empírica** (\hat{\Phi}(z)) y compararla con la teórica:
   [
   \Phi(z) = \frac{1}{2} \left[1 + \operatorname{erf}\left(\frac{z}{\sqrt{2}}\right)\right]
   ]

3. Evaluar las métricas de error:

| Métrica  | Descripción                            | Fórmula                                             |                   |   |
| -------- | -------------------------------------- | --------------------------------------------------- | ----------------- | - |
| **MAE**  | Error absoluto medio                   | ( \frac{1}{N_z} \sum                                | \hat{\Phi} - \Phi | ) |
| **RMSE** | Error cuadrático medio                 | ( \sqrt{\frac{1}{N_z} \sum (\hat{\Phi} - \Phi)^2} ) |                   |   |
| **KS**   | Máxima desviación (Kolmogórov–Smirnov) | ( \max                                              | \hat{\Phi} - \Phi | ) |

4. Analizar la **evolución de los errores** al aumentar ( n ).
   Se espera que decrezcan aproximadamente como ( 1/\sqrt{n} ).

#### 2. Validación del generador uniforme

El algoritmo Ziggurat depende de un generador uniforme ( U(0,1) ).
Por ello, se comprueba que:

* El **periodo del generador** sea suficientemente grande (p. ej. ( 2^{19937} - 1 ) en Mersenne Twister).
* El tiempo medio por muestra sea competitivo con el generador normal nativo de NumPy.

Estas medidas garantizan **independencia**, **ausencia de correlación** y **eficiencia numérica**.

---

### 📈 Resultados esperados

* La **media empírica** cercana a 0 y la **varianza** próxima a 1.
* Los errores (MAE, RMSE, KS) del orden de:

  * (10^{-3}) para ( n = 10^4 )
  * (10^{-4}) o menores para ( n = 10^6 )
* Tiempos de generación bajos (≈ microsegundos por muestra en MT19937).
* Ningún patrón visible en histogramas ni autocorrelaciones.

---

### ✅ Conclusión

El análisis empírico permite verificar la **correctitud estadística** del algoritmo Ziggurat y la **idoneidad del generador uniforme** utilizado.
A través de la comparación entre (\hat{\Phi}(z)) y (\Phi(z)), y la medición de errores y varianzas, se demuestra que la implementación reproduce con precisión la distribución normal estándar y mantiene estabilidad incluso para tamaños muestrales elevados.

---

¿Quieres que te lo adapte a un formato tipo “apartado de memoria de TFG” (con estilo más formal y narrativo, pero manteniendo las mismas secciones)?
